In [1]:
import os
import sys
import warnings

import scipy
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

# import febrisk
lib_path = os.path.join(os.path.abspath('.'), '..\\..')
sys.path.append(lib_path)
from febrisk.performance import cal_return, update_weights, cal_return_attribution, cal_risk_attribution
from febrisk.risk import expected_shortfall
from febrisk.simulation import CopulaSimulator

warnings.filterwarnings('ignore')
sns.set_style("darkgrid")

# Problem 1
Part 1 – Calculate the maximum SR portfolio using the method from last week’s homework for the following stocks: AAPL, MSFT, BRK-B, CSCO, and JNJ.  
Use the returns from the end of the history (1-14) until the end of February.  
Calculate the Ex-Post Return Attribution for each Stock.  

In [2]:
ff_3 = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
ff_4th = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
ff_4 = ff_3.join(ff_4th, how='right') / 100  # turn pct values into actual numbers
all_rets = pd.read_csv('DailyReturn.csv', parse_dates=['Date']).set_index('Date')
stocks = ['AAPL', 'MSFT' ,'BRK-B', 'CSCO', 'JNJ']
factors = ['Mkt-RF', 'SMB', 'HML', 'Mom']
reg_dataset = all_rets[stocks].join(ff_4)  # dataset for regression


# 1. calculate arithmetic E(r) in past 10 years
avg_factor_rets = ff_4.loc['2012-1-14':'2022-1-14'].mean(axis=0)
avg_daily_rets = pd.Series(dtype='float64')
betas = np.empty((len(stocks), len(factors)))
for i, stock in enumerate(stocks):
    # calculate betas
    # r_stock - r_f = alpha + sum(beta_i * factor_i) + error
    model = sm.OLS(reg_dataset[stock] - reg_dataset['RF'], sm.add_constant(reg_dataset[factors]))
    result = model.fit()
    betas[i] = result.params[1:]

    # E(r_stock) = alpha + sum(beta_i * E(factor_i)) + E(r_f)
    # assume alpha = 0
    avg_daily_rets[stock] = (result.params[factors] * avg_factor_rets[factors]).sum() \
                            + avg_factor_rets['RF'] 
    

# 2. geometric annual returns: mean and covariance
geo_means = np.log(1 + avg_daily_rets)*255  
geo_covariance = np.log(1 + all_rets[stocks]).cov()*255     


init_weights = np.array([0.10075988181538116, 0.20950981862533438,
                         0.43839111238558587, 0.08118475735284336,
                         0.17015442982085532])

In [3]:
updated_prices = pd.read_csv("updated_prices.csv", parse_dates=['Date'])
updated_rets = cal_return(updated_prices.set_index('Date')[stocks], method='arithmetic')
updated_weights = update_weights(init_weights, updated_rets.values)
weighted_rets = updated_weights * updated_rets.values

In [4]:
cal_return_attribution(weighted_rets)

array([-0.00458892, -0.0070828 , -0.00369351, -0.00754389, -0.00215082])

In [5]:
cal_risk_attribution(weighted_rets)

array([0.00156469, 0.00315964, 0.00469129, 0.00089426, 0.00140861])

# Problem 2  
Using the same data as Problem 1, attribute realized risk and return to the Fama French 3+Momentum model. Report the residual total as Portfolio Alpha.


In [6]:
updated_ff_3 = pd.read_csv('updated_F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
updated_ff_4th = pd.read_csv('updated_F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
updated_ff_4 = updated_ff_3.join(updated_ff_4th, how='right') / 100  # turn pct values into actual numbers
updated_factor_rets = pd.concat([ff_4, updated_ff_4]).loc[updated_rets.index, factors]

In [7]:
updated_factor_weights = updated_weights @ betas
weighted_factor_rets = updated_factor_weights * updated_factor_rets.values
resid_rets = weighted_rets.sum(axis=1) - weighted_factor_rets.sum(axis=1)
weighted_factor_rets = np.hstack((weighted_factor_rets, resid_rets[:, np.newaxis]))

In [8]:
cal_return_attribution(weighted_factor_rets)

array([-0.0445635 ,  0.00010344,  0.00242659, -0.00060942,  0.01758294])

In [9]:
cal_risk_attribution(weighted_factor_rets)

array([ 0.00977524, -0.00010896, -0.00071545,  0.00012717,  0.00264049])

# Problem 3
Using the same data as Problem 1 and assuming a 0 mean return, fit a t distribution to each stock return series. Simulate the system using a Gaussian Copula. Find the Risk Parity portfolio using ES as the risk measure.

In [10]:
def cal_pfl_volatility(weights, covariance):
    return np.sqrt(weights @ covariance @ weights.T)


def cal_component_std(weights, covariance):
    pfl_vol = cal_pfl_volatility(weights, covariance)
    return weights * (covariance @ weights.T) / pfl_vol


def cal_component_std_sse(weights, covariance, budget=None):
    if not budget:
        budget = np.ones_like(weights)

    component_std = cal_component_std(weights, covariance) / budget
    dev = component_std - np.mean(component_std)
    return dev @ dev.T


def gen_risk_parity_portfolio_on_std(covariance, budget=None):
    nassets = covariance.shape[0]
    
    x0 = np.array([1 / nassets for _ in range(nassets)])
    constraints = {'type':'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = ((0, 1) for _ in range(nassets))
    results = scipy.optimize.minimize(lambda w: 1e5*cal_component_std_sse(w, covariance, budget), 
                                      x0=x0, bounds=bounds, constraints=constraints)
    return results.x

std_rp_weights = gen_risk_parity_portfolio_on_std(geo_covariance.values)
std_rp_weights

array([0.15420278, 0.14256632, 0.26515987, 0.15078634, 0.28728468])

In [12]:
def cal_pfl_es(weights, returns):
    return expected_shortfall(returns @ weights.T)


def cal_component_es(weights, returns, delta=1e-6):
    nassets = len(weights)
    es = cal_pfl_es(weights, returns)
    component_es = np.empty(nassets, dtype=float)

    for i in range(nassets):
        weight_i = weights[i]
        weights[i] += delta
        component_es[i] = weight_i * (cal_pfl_es(weights, returns) - es) / delta
        weights[i] -= delta
    
    return component_es


def cal_component_es_sse(weights, returns, budget=None, delta=1e-6):
    if not budget:
        budget = np.ones_like(weights)
    component_es = cal_component_es(weights, returns, delta) / budget
    dev = component_es - np.mean(component_es)
    return dev @ dev.T


def gen_risk_parity_portfolio_on_es(returns, budget=None):
    nassets = returns.shape[1]
    x0 = np.array([1 / nassets for _ in range(nassets)])
    constraints = {'type':'eq', 'fun': lambda w: np.sum(w) - 1}
    bounds = ((0, 1) for _ in range(nassets))
    results = scipy.optimize.minimize(lambda w: 1e5*cal_component_es_sse(w, returns, budget), 
                                      x0=x0, bounds=bounds, constraints=constraints)
    return results.x


dists = []
for stock in stocks:
    df, loc, scale = scipy.stats.t.fit(all_rets[stock])
    dists.append(scipy.stats.t(df=df, loc=loc, scale=scale))
    
copula = CopulaSimulator(all_rets[stocks].values, dists)
sim_rets = copula.simulate(5000)
gen_risk_parity_portfolio_on_es(sim_rets)

array([0.16375984, 0.12566433, 0.29775739, 0.1390347 , 0.27378374])